In [ ]:
import arcpy
import os

def spatial_join_by_matching(polylines_gdb, points_gdb, output_gdb):
    """
    Perform spatial join for matching polyline and point feature classes from two geodatabases.

    Parameters:
    polylines_gdb (str): Path to the geodatabase containing polylines.
    points_gdb (str): Path to the geodatabase containing points.
    output_gdb (str): Path to the output geodatabase.
    """
    # Ensure the output geodatabase exists
    if not arcpy.Exists(output_gdb):
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))
        arcpy.AddMessage("Created output geodatabase at {}".format(output_gdb))

    # Get a list of feature classes in the polyline geodatabase
    arcpy.env.workspace = polylines_gdb
    polyline_feature_classes = arcpy.ListFeatureClasses()

    # Loop through each polyline feature class
    for polyline_fc in polyline_feature_classes:
        # Check if a corresponding point feature class exists in the points geodatabase
        points_fc = os.path.join(points_gdb, polyline_fc)  # Matching point feature class name
        if arcpy.Exists(points_fc):
            arcpy.AddMessage("Processing: {} with {}".format(polyline_fc, polyline_fc))

            # Define output feature class path
            output_fc = os.path.join(output_gdb, "SpatialJoin_{}".format(polyline_fc))

            # Perform spatial join
            arcpy.SpatialJoin_analysis(
                target_features=os.path.join(polylines_gdb, polyline_fc),
                join_features=points_fc,
                out_feature_class=output_fc,
                join_type="KEEP_COMMON",  # Options: KEEP_COMMON, KEEP_ALL
                match_option="INTERSECT"  # Match criteria (INTERSECT, WITHIN, CONTAINS, etc.)
            )
            arcpy.AddMessage("Spatial join completed: {}".format(output_fc))
        else:
            arcpy.AddMessage("Matching point feature class not found for: {}".format(polyline_fc))

    arcpy.AddMessage("Processing complete for all feature classes.")

# Main block to set up the geoprocessing tool
if __name__ == "__main__":
    # Define input parameters
    polylines_gdb = arcpy.GetParameterAsText(0)
    points_gdb = arcpy.GetParameterAsText(1)
    output_gdb = arcpy.GetParameterAsText(2)

    # Call the spatial join function
    spatial_join_by_matching(polylines_gdb, points_gdb, output_gdb)